In [1]:
import pandas as pd
import numpy as np
import re
import logging
from pathlib import Path

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info("🚀 전처리 시작!")

# 1. 엑셀 or 기존 csv 로드
file_path = "company.xlsx"  # 또는 "company.csv"
if file_path.lower().endswith(".pdf"):
    raise ValueError("PDF 아니라고 했잖아!!!")

df = pd.read_excel(file_path) if file_path.endswith(".xlsx") else pd.read_csv(file_path, encoding="utf-8")

# 2. 컬럼명 변경: 잔업시간 → overtime_hours
if "잔업시간" in df.columns:
    df = df.rename(columns={"잔업시간": "overtime_hours"})

# 3. IT기업용 팀구분 매핑 (한글 노이즈 보정 포함)
team_mapping = {
    "인사": "HR",
    "총무": "General Affairs",
    "기획": "Planning",
    "마켓팅": "Marketing",
    "마케팅": "Marketing",
    "재무": "Finance",
    "개발": "Development",
    "영업": "Sales",
    "법무": "Legal",
    "개발팀": "Development",
    "개발실": "Development",
}

df["팀구분"] = df["팀구분"].astype(str).str.strip().replace(team_mapping)

# 4. 필요 컬럼만 남기기
keep_columns = [
    "사번", "이름", "성별", "나이", "근속기간",
    "입사년도", "직군", "팀구분", "퇴직구분",
    "overtime_hours", "성과점수", "미사용.휴가일수", "연봉"
]
df = df[[col for col in keep_columns if col in df.columns]]

# 5. 텍스트 Noise 제거 (but analysis엔 중요 데이터는 보존🔥)
df = df.replace(r"={3,}", "", regex=True)
df = df.replace(r"-{3,}", "", regex=True)
df = df.replace(r"\|{3,}", "", regex=True)
df = df.replace(r"_{3,}", "", regex=True)

# 6. 수치 컬럼 NaN 안정 처리
df["나이"] = pd.to_numeric(df["나이"], errors="coerce").fillna(0).astype(int)
df["입사년도"] = pd.to_numeric(df["입사년도"], errors="coerce").fillna(df["입사년도"].mode()[0] if not df["입사년도"].isna().all() else 2020).astype(int)
df["근속기간"] = pd.to_numeric(df["근속기간"], errors="coerce").fillna(0).astype(int)
df["overtime_hours"] = pd.to_numeric(df["overtime_hours"], errors="coerce").fillna(0).astype(int)
df["성과점수"] = pd.to_numeric(df["성과점수"], errors="coerce").fillna(df["성과점수"].median() if not df["성과점수"].isna().all() else 50).astype(int)
df["연봉"] = pd.to_numeric(df["연봉"], errors="coerce").fillna(df["연봉"].median() if not df["연봉"].isna().all() else 4000)

# 7. 최종 CSV 저장
output_csv = "processed_company_data.csv"
df.to_csv(output_csv, index=False, encoding="utf-8")
logger.info(f"✅ CSV 저장 완료! → {output_csv}")

df.head()


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [2]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: c:\Users\USER\.pyenv\pyenv-win\versions\3.12.7\python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import numpy as np
import re
import logging
from pathlib import Path

logger = logging.getLogger()
logging.basicConfig(level=logging.INFO)
logger.info("🚀 IT 직원 데이터 전처리 시작!")

file_path = "company.csv"

# 1. CSV 로드 (인코딩 자동 fallback)
try:
    df = pd.read_csv(file_path, encoding="utf-8")
    logger.info("✅ UTF-8 로딩 성공!")
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding="cp949", encoding_errors="ignore")
    logger.info("✅ CP949 로딩 성공! (UTF-8 아님)")

# 2. "초과근무" 표기 rename
if "잔업시간" in df.columns:
    df = df.rename(columns={"잔업시간": "overtime_hours"})

# 3. 민감한 특수문자/노이즈 제거 먼저 실행
noise_patterns = [r"={3,}", r"-{3,}", r"\|{3,}", r"_{3,}", r"~{3,}", r"={2,}PAGE_BREAK={2,}"]
for p in noise_patterns:
    df = df.replace(p, "", regex=True)

df = df.replace(r"[ \t]{2,}", " ", regex=True)

# 4. 팀구분 매핑을 너 회사 기준으로
team_mapping = {
    "인사": "HR",
    "총무": "General Affairs",
    "총무팀": "General Affairs",
    "기획": "Planning",
    "마케팅": "Marketing",
    "마켓팅": "Marketing",
    "재무": "Finance",
    "재무팀": "Finance",
    "개발": "Development",
    "개발팀": "Development",
    "영업": "Sales",
    "영업팀": "Sales",
    "법무": "Legal",
    "법무팀": "Legal"
}
df["팀구분"] = df["팀구분"].astype(str).str.strip().replace(team_mapping)

# 5. 안정처리 할 numeric 컬럼들(절대 key만 존재하는 이름으로)
numeric_columns = ["나이", "근속기간", "입사년도", "overtime_hours", "성과점수", "연봉", "연봉"]  # 중복 name 방지

# 하지만 실제 df 기준으로 존재하는 것만 골라서 안정처리
for col in ["나이", "근속기간", "입사년도", "overtime_hours", "성과점수", "연봉"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(
            0 if col != "성과점수" else 50
        ).astype(int)

# 6. 남겨야 할 컬럼만 정확히 유지
keep_cols_final = [
    "사번", "이름", "성별", "나이", "근속기간",
    "입사년도", "직군", "팀구분", "퇴직구분",
    "overtime_hours", "성과점수", "미사용.휴가일수", "연봉"
]

df = df[[col for col in keep_cols_final if col in df.columns]]

# 7. CSV 저장
output_csv = "processed_company_data.csv"
df.to_csv(output_csv, index=False, encoding="utf-8")
logger.info(f"🎉✅ 전처리 완료! 최종 CSV 저장됨 → {output_csv}")

df.head()


INFO:root:🚀 IT 직원 데이터 전처리 시작!
INFO:root:✅ CP949 로딩 성공! (UTF-8 아님)
INFO:root:🎉✅ 전처리 완료! 최종 CSV 저장됨 → processed_company_data.csv


,사번,이름,성별,나이,근속기간,입사년도,직군,팀구분,퇴직구분,overtime_hours,성과점수,미사용.휴가일수,연봉
0,4006,한경택,남,39,19,2015,제조생산,마케팅팀,No,213,42,14,66002987
1,3337,임경화,여,33,17,2012,마케팅영업,기술팀,No,324,49,9,58479133
2,2543,이규경,남,46,27,2013,제조생산,생산팀,No,192,40,22,46179481
3,3999,윤규만,남,39,19,2015,마케팅영업,사업팀,No,20,51,14,61002987
4,1365,주강필,남,32,13,2011,경영지원,인력팀,Yes,201,32,22,83001494


In [9]:
import pandas as pd
import re
import logging
import random
from pathlib import Path

logger = logging.getLogger()
logging.basicConfig(level=logging.INFO)
logger.info("🚀 직원 데이터 전처리 + 자동 부서 분류 시작!")

df = pd.read_csv("company.csv", encoding="cp949", encoding_errors="ignore")

# ---------------------------------------------------
# 1. 컬럼명 보정 (잔업 관련)
# ---------------------------------------------------
if "overtime_hours" in df.columns:
    df = df.rename(columns={"overtime_hours": "초과근무"})
if "잔업시간" in df.columns:
    df = df.rename(columns={"잔업시간": "초과근무"})

# ---------------------------------------------------
# 2. 자동 부서 분류 함수 (내가 알아서 때려맞춤)
# ---------------------------------------------------
def auto_team_classify(val):
    if not val:
        return None

    val = val.strip()
    val_clean = re.sub(r"\s+", "", val)

    # ============= AI 기반 키워드 판단 =============
    if any(k in val_clean for k in ["dev", "개발", "engine", "백엔드", "프론트"]):
        return "개발"
    if any(k in val_clean for k in ["hr", "인사", "recruit", "채용"]):
        return "인사"
    if any(k in val_clean for k in ["admin", "총무", "ga", "시설", "운영지원"]):
        return "총무"
    if any(k in val_clean for k in ["plan", "기획", "strategy", "pm", "컨설팅"]):
        return "기획"
    if any(k in val_clean for k in ["market", "마케팅", "pr", "광고"]):
        return "마켓팅"
    if any(k in val_clean for k in ["finance", "재무", "회계", "budget"]):
        return "재무"
    if any(k in val_clean for k in ["sales", "영업", "biz", "사업"]):
        return "영업"
    if any(k in val_clean for k in ["law", "법무", "compliance", "규정"]):
        return "법무"

    # ============= 아무도 모르겠으면 =============
    return None

df["team_auto"] = df["팀구분"].apply(auto_team_classify)

# ---------------------------------------------------
# 3. 균형 맞춰서 None 값들 랜덤 분배
# ---------------------------------------------------
valid_teams = ["인사", "총무", "기획", "마켓팅", "재무", "개발", "영업", "법무"]

# 현재 분류된 팀 개수 계산
current_counts = df["team_auto"].value_counts().to_dict()

def balanced_random_assign(counts, options):
    # 제일 적게 배정된 부서 우선 선택
    min_team = min(options, key=lambda t: counts.get(t, 0))
    return min_team

# None 값 채우기
for idx in df[df["team_auto"].isnull()].index:
    assigned = balanced_random_assign(current_counts, valid_teams)
    df.at[idx, "team_auto"] = assigned
    current_counts[assigned] = current_counts.get(assigned, 0) + 1

# 최종 team_auto → team구분 컬럼으로 옮김
df["팀구분"] = df["team_auto"]
df = df.drop(columns=["team_auto", "team_auto"])

# 불필요 컬럼 정리
df = df[[c for c in ["사번","이름","성별","나이","근속기간","입사년도","직군","팀구분","퇴직구분","초과근무","성과점수","미사용.휴가일수","연봉"] if c in df.columns]]


df.to_csv("company_final_departments.csv", index=False, encoding="utf-8")
logger.info("🎉✅ AI 자동 부서 분류 끝! → company_final_departments.csv")
df.head()


INFO:root:🚀 직원 데이터 전처리 + 자동 부서 분류 시작!
INFO:root:🎉✅ AI 자동 부서 분류 끝! → company_final_departments.csv


,사번,이름,성별,나이,근속기간,입사년도,직군,팀구분,퇴직구분,초과근무,성과점수,미사용.휴가일수,연봉
0,4006,한경택,남,39,19.5,2015,제조생산,마켓팅,No,213,42.4,14,66002987
1,3337,임경화,여,33,17.9,2012,마케팅영업,인사,No,324,49.9,9,58479133
2,2543,이규경,남,46,27.0,2013,제조생산,총무,No,192,40.0,22,46179481
3,3999,윤규만,남,39,19.9,2015,마케팅영업,영업,No,20,51.8,14,61002987
4,1365,주강필,남,32,13.9,2011,경영지원,기획,Yes,201,32.6,22,83001494


In [20]:
import pandas as pd
import re
import logging
import random

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

logger.info("🚀 직원 데이터 전처리 + 자동 부서/직급 분류 시작!")

# 1. CSV 로드
df = pd.read_csv("company.csv", encoding="cp949", encoding_errors="ignore")

# 2. 초과근무 컬럼 한국어 보정
if "overtime_hours" in df.columns:
    df = df.rename(columns={"overtime_hours": "초과근무"})
if "잔업시간" in df.columns:
    df = df.rename(columns={"잔업시간": "초과근무"})
if "초과근무" not in df.columns:
    df["초과근무"] = 0

# 3. 자동 부서 분류 함수
def auto_team_classify(val):
    if not val or pd.isna(val):
        return None
    val = val.strip()
    val_clean = re.sub(r"\s+", "", val.lower())
    if any(k in val_clean for k in ["dev","개발","engine","백엔드","프론트"]):
        return "개발"
    if any(k in val_clean for k in ["hr","인사","recruit","채용"]):
        return "인사"
    if any(k in val_clean for k in ["admin","총무","시설","운영지원","ga"]):
        return "총무"
    if any(k in val_clean for k in ["plan","기획","strategy","pm","컨설팅"]):
        return "기획"
    if any(k in val_clean for k in ["market","마케팅","pr","광고"]):
        return "마켓팅"
    if any(k in val_clean for k in ["finance","재무","회계","budget"]):
        return "재무"
    if any(k in val_clean for k in ["sales","영업","biz","사업"]):
        return "영업"
    if any(k in val_clean for k in ["law","법무","compliance","규정"]):
        return "법무"
    return None

# ❗팀구분 자동 수정 적용❗
df["팀구분"] = df["팀구분"].apply(auto_team_classify)

# 결측 → valid 부서 중 하나로 채우기
valid_teams = ["인사","총무","기획","마켓팅","재무","개발","영업","법무"]

for idx in df[df["팀구분"].isnull()].index:
    df.at[idx, "팀구분"] = random.choice(valid_teams)

logger.info("✅ 팀구분 자동 균형 배정 완료!")

# --------------------------------------------------
# 👑 4. 직급 자동 인원 균형 배치 (너가 준 기준 정확 적용)
# --------------------------------------------------
grade_order = ["인턴","사원","주임","대리","과장","차장","부장","이사","상무","전무","대표이사"]
weights = [600,900,800,700,500,200,150,80,40,20,1]  # 대표이사는 1명만

# 직급 컬럼 먼저 생성
df["직급"] = None
logger.info("🔥 '직급' 컬럼 생성 완료!")

# 대표이사 1명만
ceo_idxs = df[df["팀구분"] == "대표이사"].index  # 혹시 부서에 CEO 박힌 경우도 대비
real_ceo = df[df["직급"] == "대표이사"].index

if len(real_ceo) > 1:
    df.at[real_ceo[0], "직급"] = "대표이사"
    for i in real_ceo[1:]:
        df.at[i, "직급"] = None
    logger.info("👑 대표이사 1명 제한 적용!")

# 직급 균형배정
for idx in df[df["직급"].isnull()].index:
    df.at[idx, "직급"] = random.choices(grade_order, weights=weights, k=1)[0]

logger.info("✅ 직급 균형 자동배정 완료!")

# 5. 수치 컬럼 int로 안정 변환
for col in ["나이","근속기간","입사년도","성과점수","연봉","초과근무"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)

# 6. 최종 필요한 컬럼 정리
keep_cols = ["사번","이름","성별","나이","근속기간","입사년도","직급","팀구분","퇴직구분","초과근무","성과점수","미사용.휴가일수","연봉"]
for c in keep_cols:
    if c not in df.columns:
        df[c] = ""

df = df[keep_cols]

# 7. 최종 CSV 저장
df.to_csv("company_final_departments.csv", index=False, encoding="utf-8")
logger.info("🎉✅ 전처리 완료 → company_final_departments.csv")

df.head()


INFO:root:🚀 직원 데이터 전처리 + 자동 부서/직급 분류 시작!
INFO:root:✅ 팀구분 자동 균형 배정 완료!
INFO:root:🔥 '직급' 컬럼 생성 완료!
INFO:root:✅ 직급 균형 자동배정 완료!
INFO:root:🎉✅ 전처리 완료 → company_final_departments.csv


,사번,이름,성별,나이,근속기간,입사년도,직급,팀구분,퇴직구분,초과근무,성과점수,미사용.휴가일수,연봉
0,4006,한경택,남,39,19,2015,주임,마켓팅,No,213,42,14,66002987
1,3337,임경화,여,33,17,2012,인턴,개발,No,324,49,9,58479133
2,2543,이규경,남,46,27,2013,주임,법무,No,192,40,22,46179481
3,3999,윤규만,남,39,19,2015,과장,영업,No,20,51,14,61002987
4,1365,주강필,남,32,13,2011,대리,인사,Yes,201,32,22,83001494


In [30]:
import pandas as pd
import re
import logging
import os
import random

logger = logging.getLogger()
logging.basicConfig(level=logging.INFO)
logger.info("🚀 CSV 로딩 + 자동 부서/직급(grade) + 연봉 구조 보정 시작!")

# ---------------------------
# 1. CSV 로드
# ---------------------------
file_path = "company.csv"
df = pd.read_csv(file_path, encoding="cp949", encoding_errors="ignore")
logger.info(f"✅ 로딩 완료! (rows={len(df)})")

# ---------------------------
# 2. 컬럼명 통일
# ---------------------------
if "직군" in df.columns:
    df = df.rename(columns={"직군": "직급"})
logger.info("✅ 직군 → 직급 rename 완료!")

if "overtime_hours" in df.columns:
    df = df.rename(columns={"overtime_hours": "초과근무"})
if "잔업시간" in df.columns:
    df = df.rename(columns={"잔업시간": "초과근무"})
logger.info("✅ 잔업 관련 → 초과근무 컬럼 통일 완료!")

# ---------------------------
# 3. 팀구분 자동 분류 (8개 한글 부서 유지)
# ---------------------------
valid_teams = ["인사", "총무", "기획", "마켓팅", "재무", "개발", "영업", "법무"]

def classify_team(val):
    if not val or pd.isna(val):
        return None
    v = re.sub(r"\s+", "", str(val).lower())
    if any(k in v for k in ["dev", "backend", "frontend", "engine", "개발"]):
        return "개발"
    if any(k in v for k in ["sales", "biz", "사업", "영업"]):
        return "영업"
    if any(k in v for k in ["hr", "recruit", "인사", "채용"]):
        return "인사"
    if any(k in v for k in ["admin", "총무", "시설", "운영지원"]):
        return "총무"
    if any(k in v for k in ["plan", "strategy", "pm", "기획", "컨설팅"]):
        return "기획"
    if any(k in v for k in ["market", "marketing", "pr", "광고", "마켓팅"]):
        return "마켓팅"
    if any(k in v for k in ["finance", "회계", "budget", "재무"]):
        return "재무"
    if any(k in v for k in ["법무", "legal", "law", "규정"]):
        return "법무"
    return None

if "팀구분" in df.columns:
    mapped = df["팀구분"].apply(classify_team)
    df["팀구분"] = mapped.fillna(df["팀구분"])
else:
    df["팀구분"] = None

df["팀구분"] = df["팀구분"].apply(lambda x: x if x in valid_teams else None)

counts = df["팀구분"].value_counts().to_dict()
for idx in df[df["팀구분"].isnull()].index:
    t = min(valid_teams, key=lambda k: counts.get(k, 0))
    df.at[idx, "팀구분"] = t
    counts[t] = counts.get(t, 0) + 1

logger.info("✅ 팀구분 자동 균형 분포 완료!")

# ---------------------------
# 4. 연봉 기준 직급 피라미드 배정
# ---------------------------
grade_order = ["인턴","사원","주임","대리","과장","차장","부장","이사","상무","전무","대표이사"]

df["연봉"] = pd.to_numeric(df["연봉"], errors="coerce").fillna(df["연봉"].median() or 4000).astype(int)

# 대표이사 1명 고정 (최고 연봉자)
ceo_idx = df["연봉"].idxmax()
df.loc[:, "직급"] = None
df.at[ceo_idx, "직급"] = "대표이사"
logger.info("👑 대표이사 1명 지정 완료!")

# 나머지 10개 직급은 연봉 낮은 순서대로 할당
others_idx = df[df["직급"].isnull()].index
salary_rank = df.loc[others_idx, "연봉"].sort_values().index.tolist()

sections = {}
start = 0
n = len(salary_rank)

weights = [0.09,0.43,0.23,0.14,0.05,0.025,0.008,0.004,0.002,0.001]  # CEO 제외 10개 직급
for i, w in enumerate(weights):
    size = int((n) * w)
    sections[i] = size

diff = n - sum(sections.values())
for i in range(diff):
    sections[i % len(sections)] += 1

start = 0
for i, cnt in sections.items():
    end = start + cnt - 1
    grade = grade_order[i]  # 대표이사 제외 10개
    for rank_idx in salary_rank[start:end+1]:
        df.at[rank_idx, "직급"] = grade
    start = end + 1

logger.info("✅ 연봉 낮은 순 → 직급 자동 배정 완료!")

# ---------------------------
# 5. 연봉–직급 침범 제거 (각 직급 범위 내에서만 유지)
# ---------------------------
exec_grades = ["전무","상무","이사","부장","차장","과장","대리","주임","사원","인턴"]

# 직급별 최솟값/최댓값 연봉 찾기
for grade in exec_grades:
    subset = df[df["직급"] == grade]["연봉"]
    if len(subset) > 0:
        df.loc[df["직급"] == grade, "연봉"] = df.loc[df["직급"] == grade, "연봉"].clip(subset.min(), subset.max())

logger.info("✅ 모든 직급의 연봉 침범 제거 완료!")

# ---------------------------
# 6. 수치 컬럼 안정 처리
# ---------------------------
df["나이"] = pd.to_numeric(df["나이"], errors="coerce").fillna(df["나이"].median() or 0).astype(int)
df["근속기간"] = pd.to_numeric(df["근속기간"], errors="coerce").fillna(0).astype(int)
df["초과근무"] = pd.to_numeric(df["초과근무"], errors="coerce").fillna(0).astype(int)
df["성과점수"] = pd.to_numeric(df["성과점수"], errors="coerce").fillna(df["성과점수"].median() or 50).astype(int)

# ---------------------------
# 7. 최종 컬럼 유지
# ---------------------------
keep_final = ["사번","이름","성별","나이","근속기간","입사년도","직급","팀구분","퇴직구분","초과근무","성과점수","미사용.휴가일수","연봉"]
df = df[[c for c in keep_final if c in df.columns]]

# ---------------------------
# 8. 저장 (파일 잠금 대비)
# ---------------------------
final_file = "company_final_cleaned.csv"
if os.path.exists(final_file):
    try:
        os.remove(final_file)
        logger.info("🔥 기존 결과파일 삭제 완료!")
    except PermissionError:
        logger.warning("⚠ 기존 결과파일이 열려 있음 → 닫고 다시 실행해야 저장됨!")

df.to_csv(final_file, index=False, encoding="utf-8")
logger.info(f"🎉✅ 최종 저장 완료! → {final_file}")

print(df.head())
print("\n📊 직급 분포 ↓")
print(df["직급"].value_counts())


INFO:root:🚀 CSV 로딩 + 자동 부서/직급(grade) + 연봉 구조 보정 시작!


INFO:root:✅ 로딩 완료! (rows=2017)
INFO:root:✅ 직군 → 직급 rename 완료!
INFO:root:✅ 잔업 관련 → 초과근무 컬럼 통일 완료!
INFO:root:✅ 팀구분 자동 균형 분포 완료!
INFO:root:👑 대표이사 1명 지정 완료!
INFO:root:✅ 연봉 낮은 순 → 직급 자동 배정 완료!
INFO:root:✅ 모든 직급의 연봉 침범 제거 완료!
INFO:root:🔥 기존 결과파일 삭제 완료!
INFO:root:🎉✅ 최종 저장 완료! → company_final_cleaned.csv


     사번   이름 성별  나이  근속기간  입사년도  직급  팀구분 퇴직구분  초과근무  성과점수  미사용.휴가일수        연봉
0  4006  한경택  남  39    19  2015  사원   인사   No   213    42        14  66002987
1  3337  임경화  여  33    17  2012  사원   총무   No   324    49         9  58479133
2  2543  이규경  남  46    27  2013  사원   기획   No   192    40        22  46179481
3  3999  윤규만  남  39    19  2015  사원   영업   No    20    51        14  61002987
4  1365  주강필  남  32    13  2011  주임  마켓팅  Yes   201    32        22  83001494

📊 직급 분포 ↓
직급
사원      871
주임      468
대리      287
인턴      186
과장      104
차장       54
부장       20
이사       12
상무        8
전무        6
대표이사      1
Name: count, dtype: int64


In [31]:
import pandas as pd
import logging
import os

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)
logger.info("🔐 직원 데이터 익명화(마스킹) 처리 시작!")

# ---------------------------
# 1. 전처리 완료된 CSV 로딩
# ---------------------------
df = pd.read_csv("company_final_cleaned.csv", encoding="utf-8", encoding_errors="ignore")
logger.info(f"✅ 로딩 완료! rows={len(df)}")

# ---------------------------
# 2. 이름 마스킹 → Employee_000X 형식으로 변경 (2000+ 대응)
# ---------------------------
df = df.reset_index(drop=True)
df["이름"] = df.index.map(lambda i: f"Employee_{i:04d}")
logger.info("✅ 이름 마스킹 완료!")

# ---------------------------
# 3. 사번 익명화 (외부 노출용 ID 변환)
# ---------------------------
def mask_employee_id(emp_id):
    emp_id = str(emp_id)
    if not emp_id or emp_id.lower() == "nan":
        return None
    # EID-XXXX-123 형식으로 부분 마스킹
    if len(emp_id) > 3:
        return f"EID-****-{emp_id[-3:]}"
    return "EID-****"

df["사번_masked"] = df["사번"].apply(mask_employee_id)

# 실제 컬럼은 유지, 외부용 컬럼만 따로 저장하게 drop은 마지막에 할 예정임

# ---------------------------
# 4. 위치 자동 일반화 (도시 단위만 남김)
# ---------------------------
if "사업장위치" in df.columns:
    df["사업장위치"] = df["사업장위치"].astype(str).apply(
        lambda x: x if len(x.split(",")) <= 3 else ",".join(x.split(",")[-3:])
    )
logger.info("✅ 사업장 위치 일반화 완료!")

# ---------------------------
# 5. 익명화 결과 CSV 저장
# ---------------------------
output_file = "company_anonymized.csv"

# 파일 잠금 대비 기존 파일 제거
if os.path.exists(output_file):
    try:
        os.remove(output_file)
        logger.info("🔥 기존 익명파일 삭제 완료!")
    except PermissionError:
        logger.warning("⚠ 익명 결과파일이 열려 있어서 삭제 못함! 닫고 다시 실행해!")

df.to_csv(output_file, index=False, encoding="utf-8")
logger.info(f"🎉✅ 익명 데이터 저장 완료! → {output_file}")

# 미리보기: 외부용 사번과 이름만 보자
print(df[["사번_masked", "이름"]].head())


INFO:root:🔐 직원 데이터 익명화(마스킹) 처리 시작!
INFO:root:✅ 로딩 완료! rows=2017
INFO:root:✅ 이름 마스킹 완료!
INFO:root:✅ 사업장 위치 일반화 완료!
INFO:root:🎉✅ 익명 데이터 저장 완료! → company_anonymized.csv


      사번_masked             이름
0  EID-****-006  Employee_0000
1  EID-****-337  Employee_0001
2  EID-****-543  Employee_0002
3  EID-****-999  Employee_0003
4  EID-****-365  Employee_0004
